In [1]:
import openseespy.opensees as ops
from createnodes import createnodes

In [3]:
ops.wipe()
ops.model('basic', '-ndm', 2, '-ndf', 3)
createnodes(13)
ops.getNodeTags()

[13, 14]

In [1]:
import numpy as np
from createnodes import *
a = np.zeros([2, 2, 2])
a[0, 0, :] = [1, 2]
b = a[0, 0, :]
c = test(b)
c

1.0


array([1., 2.])

In [13]:
b = 10
a = '%i%s' % (b,'01')
a = int(a)
a

1001